In [2]:
import tensorflow as tf
import datetime as dt

In [3]:
#Simple Layer
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name="train_me")
        self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable


In [4]:
simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


## First a dense (linear) layer:

In [5]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
          tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

## And then the complete model, which makes two layer instances and applies them:

In [6]:
class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.39031565 0.        ]], shape=(1, 2), dtype=float32)


In [7]:
print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x00000222FF6AF048>, <__main__.Dense object at 0x00000222D64672C8>)


In [8]:
for var in my_model.variables:
    print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.10272066, -0.58512753, -0.06615195],
       [ 1.0061566 ,  2.0000758 ,  1.6512033 ],
       [-0.7395761 , -0.02542531, -0.6074709 ]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.64518607, -0.05222085],
       [ 0.11332027,  0.04047208],
       [-0.20517106, -1.3543276 ]], dtype=float32)> 



# Waiting to create variables

In [9]:
class FlexibleDenseModule(tf.Module):
  # Note: No need for `in+features`
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    # Create variables on first call.
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [10]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.         0.34588957]], shape=(1, 2), dtype=float32)


##  Load the TensorBoard notebook extension

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
## Clear log File

In [13]:
# Clear any logs from previous runs
#rm -rf ./logs/

### Saving weights

In [14]:
#chkp_path = "my_checkpoint"
chkp_path = "E:\logs\my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'E:\\logs\\my_checkpoint'

In [15]:
#ls my_checkpoint*
#ls E:\\log\\my_checkpoint

In [16]:
ls E:\logs\*

 Volume in drive E is WorkSpace
 Volume Serial Number is ACF8-467B

 Directory of E:\logs

31-01-2021  22:31    <DIR>          .
31-01-2021  22:31    <DIR>          ..
31-01-2021  22:31               347 my_checkpoint.data-00000-of-00002
31-01-2021  22:31                80 my_checkpoint.data-00001-of-00002
31-01-2021  22:31               368 my_checkpoint.index
               3 File(s)            795 bytes
               2 Dir(s)  342,300,008,448 bytes free


In [17]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

# Overwrite Save Model

In [18]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
#new_checkpoint.restore("my_checkpoint")
new_checkpoint.restore(chkp_path)

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.        , 0.34588957]], dtype=float32)>

## Saving functions

In [19]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [20]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[1.6241103 0.       ]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[1.6241103 0.       ]
  [1.6241103 0.       ]]], shape=(1, 2, 2), dtype=float32)


# You can visualize the graph by tracing it within a TensorBoard summary.

In [21]:
# Set up logging.
stamp = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
#logdir = "logs/func/%s" % stamp
logdir = "E:/logs/func/%s" % stamp
logdir

'E:/logs/func/20210131-223144'

In [22]:
writer = tf.summary.create_file_writer(logdir)

In [23]:
# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[1.0052372 0.       ]], shape=(1, 2), dtype=float32)


## Launch TensorBoard to view the resulting trace:

In [25]:
#%tensorboard --logdir E:/logs/func

In [ ]:
#In cmd shell
#tensorboard --logdir="E:\logs\func"

In [ ]:
#%tensorboard --logdir E:/logs/func

In [ ]:
# Set tensor log dir(In cmd Shell)
#tensorboard --logdir %cd%/log/

[Tensorboard](http://localhost:6006/)

# TensorBoard

In [26]:

from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir E:/logs/func (started 8:03:31 ago; pid 13472)
  - port 6006: logdir E:\logs\func (started 7:48:08 ago; pid 17908)
  - port 6006: logdir logs/fit (started 4 days, 21:39:52 ago; pid 18388)
  - port 6006: logdir E:/logs/func (started 0:04:55 ago; pid 2184)
  - port 6006: logdir E:\AI_ML_DL (started 9:33:18 ago; pid 22156)
  - port 6006: logdir %cd%/log/ (started 8:46:56 ago; pid 22728)
  - port 6006: logdir logs/func (started 7:20:19 ago; pid 6252)


In [ ]:
notebook.display(port=6006, height=1000)